# experimental
> Various tools that I'm not so sure about that are at the bleeding edge

In [ ]:
#| default_exp experimental

In [ ]:
#|export
from typing import List, Iterable, Union
from collections import Counter
from pathlib import Path
import pickle

import pandas as pd
from pydantic import BaseModel
import langsmith
from fastcore.foundation import first, L
from fastcore.test import test_eq
from langfree.runs import (get_runs, get_output, get_input, 
                           get_params, get_functions,
                          get_feedback, client, take)

In [ ]:
#|export
class LLMRecord(BaseModel):
    "A parsed run from LangSmith."
    child_run_id:str
    parent_run_id:str
    llm_input:str
    llm_output: str
    url: str
    total_tokens:Union[int, None]
    prompt_tokens:Union[int, None]
    completion_tokens:Union[int, None]
    human_readable:bool=False
    feedback: Union[List,None] = None
    feedback_keys: Union[List,None] = None
    tags: Union[List,None] = []
    start_dt: Union[str, None] = None
    parent_url: Union[str,None] = None
    parent_id: Union[str,None] = None
    function_defs: Union[List,None] = None
    param_model_name: Union[str,None]= None
    param_n: Union[int, None] = None
    param_top_p: Union[int, None] = None
    param_temp: Union[int, None] = None
    param_presence_penalty: Union[int, None] = None
    param_freq_penalty: Union[int, None] = None
    error_categories: List[str] = []
    
    @classmethod
    def collate(cls, run:langsmith.schemas.Run, human_readable=False):
        "Collate information About A Run into a `LLMRecord`."
        error_categories = []
        if run.execution_order != 1: # this is a child run, get the parent
            crun = run
            run = client.read_run(run.parent_run_id)
            
        _cruns = client.read_run(run_id=run.id, load_child_runs=True).child_runs
        crun = _cruns[-1] if _cruns else None

    
        if crun:
            if crun.name != 'ChatOpenAI': 
                error_categories.append('Last Step Not ChatOpenAI')
                _input, _output = '', ''
            else: _input, _output = get_input(crun), get_output(crun)
                
            if 'Agent stopped due to max iterations' in _input: error_categories.append('Max Iterations')
            if _output.strip() == '': error_categories.append('No Output')
            
            params = get_params(crun)
            _feedback = get_feedback(run) # you must get feedback from the root
            
            return cls(child_run_id=str(crun.id),
                       parent_run_id=str(run.id),
                       llm_input=_input,
                       llm_output=_output,
                       url=crun.url,
                       total_tokens=crun.total_tokens,
                       prompt_tokens=crun.prompt_tokens,
                       completion_tokens=crun.completion_tokens,
                       human_readable=human_readable,
                       feedback=_feedback, 
                       feedback_keys=list(L(_feedback).attrgot('key').filter()),
                       tags=run.tags,
                       start_dt=run.start_time.strftime('%m/%d/%Y'),
                       parent_url=run.url if run else None,
                       parent_id=str(run.id) if run else None,
                       function_defs=get_functions(crun),
                       error_categories=error_categories,
                       **params)
        
    def _repr_markdown_(self):
        if not self.human_readable: return None
        feedback_str = ''
        if self.feedback:
            for f in self.feedback:
                for k,v in f.items():
                    feedback_str+=f'{k}: {v}\n'
        
        raw_markdown =  f"""
<details>
  <summary>Show/Collapse Run</summary>
  
# URLs
Child: {self.url}

Parent: {self.parent_url}

Tags: {self.tags}

# Inputs:

{self.llm_input}

# Outputs:

{self.llm_output}

# Feedback: {bool(self.feedback)}

{feedback_str}

# Error Categories: {bool(self.error_categories)}

{self.error_categories}
</details>
"""
        blockquoted_markdown = '\n'.join([f'> {line}' for line in raw_markdown.split('\n')])
        return blockquoted_markdown
    
    def show(self):
        self.human_readable=True
        return self

In [ ]:
#|hide
_root_run = client.read_run('fbfd220a-c731-46a2-87b3-e64a477824f5')

_child = client.read_run(_root_run.child_run_ids[-1])
_root_result = LLMRecord.collate(_root_run)
_child_result = LLMRecord.collate(_child)

# test that child and root runs are related
test_eq(_root_result.llm_output, _child_result.llm_output)
test_eq(_root_result.parent_run_id, _child_result.parent_run_id)

# Test case without feedback
_parent_run_no_feedback = client.read_run('87900cfc-0322-48fb-b009-33d226d73597')
_no_feedback = LLMRecord.collate(_parent_run_no_feedback)
test_eq(_no_feedback.feedback, [])

# Test case with feedback

#  ... starting with a child run
_child_w_feedback = client.read_run('f8717b0e-fb90-45cd-be00-9b4614965a2e')
_feedback = LLMRecord.collate(_child_w_feedback).feedback
assert _feedback[0]['key'] == 'Empty Response'

# #  ... starting with a parent run
_parent_w_feedback = client.read_run(_child_w_feedback.parent_run_id)
_feedback2 = LLMRecord.collate(_parent_w_feedback).feedback
test_eq(_feedback[0]['comment'],  _feedback2[0]['comment'])

## Saving & Loading A Dataset of `LLMRecord`

In [ ]:
#|export
class LLMDataset(BaseModel):
    "A collection of `LLMRecord`."
    records: List[LLMRecord]
    tags: Counter
    dates: Counter
    
    @classmethod
    def from_commit(cls, commit_id:str):
        "Create a `LLMDataset` from a commit id"
        _runs = get_runs(commit_id=commit_id)
        return cls.from_runs(_runs)
    
    @classmethod
    def from_runs(cls, runs:List[langsmith.schemas.Run]):
        "Load LLMDataset from runs."
        tag_counter=Counter()
        date_counter=Counter()
        records=[LLMRecord.collate(r, human_readable=False) for r in runs]
        for rec in records:
            if rec.tags: tag_counter.update(rec.tags)
            if rec.start_dt: date_counter.update([rec.start_dt])
        return cls(records=records, tags=tag_counter, dates=date_counter)
    
    def __len__(self):
        return len(self.records)
    
    def save(self, path:str):
        "Save data to disk."
        dest_path = Path(path)
        if not dest_path.parent.exists(): dest_path.parent.mkdir(exist_ok=True)
        with open(dest_path, 'wb') as f:
            pickle.dump(self, f)
            return dest_path
        
    def __iter__(self):
        for r in self.records: yield r
    
    @classmethod
    def load(cls, path:str):
        "Load data from disk."
        src_path = Path(path)
        with open(src_path, 'rb') as f:
            obj = pickle.load(f)
            if isinstance(obj, cls):
                return obj
            else:
                raise TypeError(f"The loaded object is not of type {cls.__name__}")
                
    def to_pandas(self):
        "Convert the `LLMDataset` to a pandas.DataFrame."
        return pd.DataFrame(L(self.records).map(dict))
    
    def to_airtable_csv(self, path:str):
        "Format a csv such that it can be exported to Airtable conveniently."
        dest_path = Path(path)
        if not dest_path.parent.exists(): dest_path.parent.mkdir(exist_ok=True)
        if dest_path.suffix != '.csv': raise Exception(f"You must name your file with a csv extension, instead got {path}")
        data = deepcopy(self)
        for r in data.records:
            r.tags = ', '.join(r.tags)
            r.feedback_keys = ', '.join(r.feedback_keys)
            r.error_categories = ', '.join(r.error_categories)
        df = pd.DataFrame(L(data.records).map(dict))
        df.to_csv(dest_path)
        return dest_path
                
    def __repr__(self):
        tags = '\n'.join([f'- {x} {y}' for x,y in self.tags.most_common(5)])
        dates = '\n'.join([f'- {x} {y}' for x,y in self.dates.most_common(5)])
        return f'LLMDataset with {len(self)} records.\n\nDates:\n{dates}\n\n5 Most common tags:\n{tags}'

In [ ]:
from langfree.runs import get_runs
_runs = get_runs(commit_id='028e4aa4')
llmdata = LLMDataset.from_runs(take(_runs, 10))

Fetching runs with this filter: and(eq(status, "success"), has(tags, "commit:028e4aa4"))


### Convert `LLMDataset` to a Pandas Dataframe

You can do this with `to_pandas()`

In [ ]:
_df = llmdata.to_pandas()
assert _df.shape[0] == 10

### Save Data

In [ ]:
#|eval: false
llmdata.save('_data/llm_data.pkl')

PosixPath('_data/llm_data.pkl')

### Load Data

In [ ]:
#|eval: false
_loaded = LLMDataset.load('_data/llm_data.pkl')